# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.chdir('..')
sys.path.append('src')

In [3]:
from pathlib import Path

import postprocessing

# Code

In [5]:
src_folder = 'input/hm/train/'
img_name = next(Path(src_folder).glob('*.tiff'))
print(img_name)

input/hm/train/54f2eec69.tiff


In [7]:
mask = postprocessing.read_and_process_img(img_name, test_inf, size=512)
mask.shape

TypeError: read_and_process_img() got an unexpected keyword argument 'size'

In [ ]:
_plot_img(mask)

# Tests

In [8]:
postprocessing.postprocess_test_folder(test_inf, src_folder, './output')

30440 22240


/home/gsm/.local/lib/python3.7/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


AttributeError: 'numpy.ndarray' object has no attribute 'unsqueeze'